Define file paths

In [1]:
import functools
import json
import time
from pathlib import Path
from typing import List

# Data analysis libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Display HTML in jupyer
from IPython.display import HTML, display

# Progress bar
from tqdm import tqdm

# Paths
input_path = Path.cwd()
working_path = Path.cwd() / "working"
cards_path = input_path / "clash-royale-cards"
assets_path = input_path / "clash-royale-assets"
games_path = input_path / "clash-royale-games"

# Matplotlib style
plt.style.use("ggplot")

Convert small portion of clash royale dataset into parquet to then apply to all files in large scale conversion

In [10]:
import pandas as pd
import json
from pathlib import Path

# Paths
input_path = Path.cwd()
working_path = Path.cwd() / "working"
cards_path = input_path / "clash-royale-cards"
games_path = input_path / "clash-royale-games"

# One example file and season
season = "20230103-20230206"
day = "20230103"

# CSV and Parquet Paths
csv_path = games_path / season / season / f"{day}.csv"
parquet_path = working_path / season / f"compact_{day}.parquet"

# Ensure directory exists
parquet_path.parent.mkdir(parents=True, exist_ok=True)

# Column Indices
INFO = [0, 1]
INFO_TEAM = [2, 3, 4]
DECK_TEAM = [5, 6, 7, 8, 9, 10, 11, 12]
INFO_OPPONENT = [13, 14, 15]
DECK_OPPONENT = [16, 17, 18, 19, 20, 21, 22, 23]

# Load Card Mappings
with open(cards_path / "cards.json") as f:
    CARDS_INDEX = {str(card["id"]): i for i, card in enumerate(json.load(f))}

# Read CSV
converters = {
    1: int,  # game_mode
    2: str,  # team tag
    3: int,  # team trophies
    4: int,  # team crowns
    **{i: CARDS_INDEX.get for i in DECK_TEAM},  # team deck
    13: str,  # opponent tag
    14: int,  # opponent trophies
    15: int,  # opponent crowns
    **{i: CARDS_INDEX.get for i in DECK_OPPONENT},  # opponent deck
}

battles = pd.read_csv(csv_path, header=None, converters=converters, parse_dates=[0])

# Store Deck as Lists
battles["team_cards"] = battles.iloc[:, DECK_TEAM].apply(lambda row: [card for card in row if card is not None], axis=1)
battles["opponent_cards"] = battles.iloc[:, DECK_OPPONENT].apply(lambda row: [card for card in row if card is not None], axis=1)

# Drop previous deck columns from original dataset
battles.drop(columns=DECK_TEAM + DECK_OPPONENT, inplace=True)

# Redefine column names without tuples
columns = [
    "info_datetime",
    "info_game_mode",
    "team_tag",
    "team_trophies",
    "team_crowns",
    "team_cards",
    "opponent_tag",
    "opponent_trophies",
    "opponent_crowns",
    "opponent_cards"
]
battles.columns = columns

# Save to Parquet
battles.to_parquet(parquet_path, engine="pyarrow", index=False)
print(f"{csv_path} was successfully converted to {parquet_path}")



/Users/bretthillyard/Desktop/CSC369/CSC369_Assignments/FinalProject/clash-royale-games/20230103-20230206/20230103-20230206/20230103.csv was successfully converted to /Users/bretthillyard/Desktop/CSC369/CSC369_Assignments/FinalProject/working/20230103-20230206/compact_20230103.parquet


Example code to find card usage rate in small subset of entire dataset

In [ ]:
import pandas as pd

# Load the sample small Parquet file
parquet_path = "working/20230103-20230206/compact_20230103.parquet"
battles = pd.read_parquet(parquet_path, engine="pyarrow")

# Count occurrences of each card in both team and opponent decks
team_card_counts = battles["team_cards"].explode().value_counts()
opponent_card_counts = battles["opponent_cards"].explode().value_counts()

# Total number of games (each game has 2 players using decks)
total_games = len(battles)

# Calculate percentage usage per card
card_usage_rate = (team_card_counts.add(opponent_card_counts, fill_value=0) / (2 * total_games)) * 100

# Convert to DataFrame for display
card_usage_df = pd.DataFrame({"Card Usage (%)": card_usage_rate}).fillna(0)

# Show top 10 most used cards
print(card_usage_df.sort_values(by="Card Usage (%)", ascending=False).head(10))


     Card Usage (%)
111       19.029282
112       11.321390
101       10.057012
100        9.772417
108        9.525876
115        8.707863
21         8.420483
11         8.266990
85         7.889076
10         7.219529


Code to format and convert all csvs into one large parquet file to query on for final analysis

In [ ]:
import pandas as pd
import json
import pyarrow.parquet as pq
import pyarrow as pa
from pathlib import Path

# Paths
input_path = Path.cwd()
working_path = Path.cwd() / "working"
cards_path = input_path / "clash-royale-cards"
games_root = input_path / "clash-royale-games"
output_parquet = working_path / "clash_royale_all_data.parquet"

# Ensure working directory exists
working_path.mkdir(parents=True, exist_ok=True)

# Column Indices
INFO = [0, 1]
INFO_TEAM = [2, 3, 4]
DECK_TEAM = [5, 6, 7, 8, 9, 10, 11, 12]
INFO_OPPONENT = [13, 14, 15]
DECK_OPPONENT = [16, 17, 18, 19, 20, 21, 22, 23]

# Load Card Mappings
with open(cards_path / "cards.json") as f:
    CARDS_INDEX = {str(card["id"]): i for i, card in enumerate(json.load(f))}

# Find all CSV files inside the directories (SORTED BY OLDEST FIRST)
csv_files = sorted(games_root.rglob("*.csv"), key=lambda x: x.parent.name)

# Group files by directory
directories = {}
for file in csv_files:
    dir_name = file.parent.name
    if dir_name not in directories:
        directories[dir_name] = []
    directories[dir_name].append(file)

# Parquet Writer Initialization
parquet_writer = None

# Process each directory
for dir_name, files in directories.items():
    for csv_file in files:
        # Read CSV in CHUNKS to avoid memory overload
        chunk_iter = pd.read_csv(csv_file, header=None, converters={
            1: int,  # game_mode
            2: str,  # team tag
            3: int,  # team trophies
            4: int,  # team crowns
            **{i: CARDS_INDEX.get for i in DECK_TEAM},  # team deck
            13: str,  # opponent tag
            14: int,  # opponent trophies
            15: int,  # opponent crowns
            **{i: CARDS_INDEX.get for i in DECK_OPPONENT},  # opponent deck
        }, parse_dates=[0], chunksize=50000)  # Adjust chunksize based on your RAM

        # Process each chunk
        for battles in chunk_iter:
            # Store deck as lists instead of one-hot encoding
            battles["team_cards"] = battles.iloc[:, DECK_TEAM].apply(lambda row: [card for card in row if card is not None], axis=1)
            battles["opponent_cards"] = battles.iloc[:, DECK_OPPONENT].apply(lambda row: [card for card in row if card is not None], axis=1)

            # Drop original deck columns
            battles.drop(columns=DECK_TEAM + DECK_OPPONENT, inplace=True)

            # Flattened Column Names (No Tuples)
            battles.columns = [
                "info_datetime",
                "info_game_mode",
                "team_tag",
                "team_trophies",
                "team_crowns",
                "team_cards",
                "opponent_tag",
                "opponent_trophies",
                "opponent_crowns",
                "opponent_cards"
            ]

            # Convert to PyArrow Table
            table = pa.Table.from_pandas(battles)

            # Write to Parquet using PyArrow's ParquetWriter
            if parquet_writer is None:
                parquet_writer = pq.ParquetWriter(output_parquet, table.schema, compression="zstd")

            parquet_writer.write_table(table)

    # Print directory name after all its files are processed
    print(f"Finished processing directory: {dir_name}")

# Close Parquet Writer
if parquet_writer is not None:
    parquet_writer.close()

print(f"\nProcessed & merged {len(csv_files)} files into {output_parquet} in chronological order.")



Finished processing directory: 20220906-20221003
Finished processing directory: 20221004-20221107
Finished processing directory: 20221108-20221205
Finished processing directory: 20221206-20230102
Finished processing directory: 20230103-20230206
Finished processing directory: 20230206-20230306
Finished processing directory: 20230306-20230403
